In [1]:
from types import MethodType
from escalerasyserpientes import EscalerasSerpientes
import copy
import numpy as np

In [2]:
es=EscalerasSerpientes([80,100],[23,37,45,67,89],-0.01)

In [3]:
#SARSA algorithm

# Variables: % ganados, pasos x episodio, alpha y gamma.
win_per = 0
prom_step = 0
alpha = 0.5
gamma = 1
epsilon = 0.1


#Para 1000 episodios de entrenamiento
for i in range(10000):
    # Inicializa las variables para cada episodio
    num_steps = 0
    state = es.states[0]
    # Seleccion accion epsilon-greedy
    if epsilon< np.random.uniform():        
        act_arg = np.array([es.q_values[(state, act)] for act in es.allowed_actions[state]])
        action = es.allowed_actions[state][np.argmax(act_arg)]
    else:
        actions = es.allowed_actions[state]
        action = actions[np.random.randint(0,2)]
    
    # Inicia el episodio
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]

        ### Tomar acción dependiendo de epsilon y de la acción con mayor recompensa
        
        # Obtengo s'
        new_state, reward, _, done = es.step(state, action, random=True)
        
        # Revisa que new_state no sea un estado terminal
        if type(new_state) is str:
            # Valor q(s',a') terminal
            q_value_next_step = 0
        else:
            #Obtengo a' de s' con epsilon greedy
            if np.random.uniform()< epsilon:     
                actions = es.allowed_actions[new_state]
                new_action = actions[np.random.randint(0,2)]
            else:
                act_arg = np.array([es.q_values[(new_state, act)] for act in es.allowed_actions[state]])
                new_action = es.allowed_actions[new_state][np.argmax(act_arg)]

            # Valor q(s',a') no terminal
            q_value_next_step = es.q_values[(new_state,new_action)]
        
        
        # Calculo de actualizacion q(s,a) <- q(s,a) + alpha*(R + gamma*q(s',a') - q(s,a))
        es.q_values[(state, action)] += alpha*(reward + gamma*q_value_next_step - es.q_values[(state,action)])
        
        # asigna a = a' y s = s'
        state = new_state
        action = new_action
        #print(action, state)
        # Numero de pasos utilizados en el episodio
        #num_steps+=1        

        # Parte que termina el episodio si se llega a algun estado terminal
        if done:
            continue_episode = False
            #if state == 'Azul':
                #win_per+=1
                
    # Promedio de pasos por episodio
    #prom_step += 1/(i+1) *(num_steps - prom_step)
    #print('-'*20)


In [4]:
SARSA_policy = {}
for i in es.states:
    #print(i)
    max_val = np.argmax([es.q_values[(i,act)] for act in es.allowed_actions[i]])
    SARSA_policy[i] = es.allowed_actions[i][max_val]
print(SARSA_policy)

{1: 'At', 2: 'At', 3: 'At', 4: 'Ad', 5: 'At', 6: 'Ad', 7: 'Ad', 8: 'Ad', 9: 'Ad', 10: 'Ad', 11: 'At', 12: 'Ad', 13: 'At', 14: 'Ad', 15: 'Ad', 16: 'Ad', 17: 'At', 18: 'At', 19: 'At', 20: 'At', 21: 'Ad', 22: 'At', 23: 'Ad', 24: 'Ad', 25: 'Ad', 26: 'At', 27: 'At', 28: 'Ad', 29: 'Ad', 30: 'Ad', 31: 'At', 32: 'At', 33: 'At', 34: 'At', 35: 'At', 36: 'At', 37: 'Ad', 38: 'Ad', 39: 'At', 40: 'Ad', 41: 'At', 42: 'Ad', 43: 'Ad', 44: 'Ad', 45: 'Ad', 46: 'Ad', 47: 'Ad', 48: 'Ad', 49: 'Ad', 50: 'Ad', 51: 'Ad', 52: 'Ad', 53: 'At', 54: 'Ad', 55: 'Ad', 56: 'Ad', 57: 'Ad', 58: 'Ad', 59: 'Ad', 60: 'At', 61: 'At', 62: 'Ad', 63: 'At', 64: 'Ad', 65: 'At', 66: 'Ad', 67: 'Ad', 68: 'Ad', 69: 'Ad', 70: 'Ad', 71: 'Ad', 72: 'Ad', 73: 'Ad', 74: 'Ad', 75: 'Ad', 76: 'Ad', 77: 'Ad', 78: 'Ad', 79: 'Ad', 80: 'Ad', 81: 'At', 82: 'At', 83: 'Ad', 84: 'At', 85: 'At', 86: 'At', 87: 'Ad', 88: 'At', 89: 'Ad', 90: 'At', 91: 'Ad', 92: 'Ad', 93: 'Ad', 94: 'Ad', 95: 'Ad', 96: 'At', 97: 'At', 98: 'Ad', 99: 'At', 100: 'Ad'}


In [5]:
# Prueba de polica en SARSA
num_episodios = 1000
win_per = 0
prom_step = 0

for i in range(num_episodios):
    num_steps = 0
    state = es.states[0]
    states = []
    
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]
            
        # Toma la accion de la politica
        action = SARSA_policy[state]
        # Da un paso en direccion de la politica
        new_state, reward, _, done = es.step(state, action, random=True)
        
        state = new_state
        states.append(state)
        num_steps+=1
        
        if done or num_steps >500:
            continue_episode = False
            if state == 'Azul':
                win_per+=1
    #print(states)
    prom_step += 1/(i+1) *(num_steps - prom_step)
    
#win_per = win_per*100/num_episodios

In [6]:
print("Promedio pasos por episodio: ",prom_step)
print("Porcentaje victorias: ", win_per*100/num_episodios)

Promedio pasos por episodio:  37.83899999999997
Porcentaje victorias:  76.8


### Q-Learning

In [7]:
#Q-learning algorithm

# Variables: % ganados, pasos x episodio, alpha y gamma.
win_per = 0
prom_step = 0
alpha = 0.5
gamma = 1


#Para 1000 episodio
for i in range(10000):
    # Inicializa las variables para cada episodio
    num_steps = 0
    state = es.states[0]
    
    # Inicia el episodio
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]

        # Tomar acción de forma epsilon-greedy
        epsilon = 0.1
        if np.random.uniform()<epsilon:
            # paso aleatorio
            A = np.random.randint(0,2)
            actions = es.allowed_actions[1]
            action = actions[A]

        else:
            # paso con accion greedy
            act_arg = np.array([es.q_values[(state, act)] for act in es.allowed_actions[state]])
            action = es.allowed_actions[state][np.argmax(act_arg)]
        
        # Obtengo s'
        new_state, reward, _, done = es.step(state, action, random=True)

        # Valor de max q(s',a). Si es terminal el estado, el valor es 0
        if type(new_state) is str:
            # Valor max q(s',a) terminal
            max_q_val = 0
        else:
            # Valor max q(s',a) no terminal
            action_arg = np.array([es.q_values[(new_state, act)] for act in es.allowed_actions[state]])
            new_action = es.allowed_actions[new_state][np.argmax(action_arg)]
            max_q_val = es.q_values[(new_state,new_action)]
        
        # Calculo de actualizacion q(s,a) <- q(s,a) + alpha*(R + gamma*max q(s',a) - q(s,a))
        es.q_values[(state, action)] += alpha*(reward + gamma*max_q_val - es.q_values[(state,action)])
        
        # asigna s = s'
        state = new_state
        #print(action, state)
        # Numero de pasos utilizados en el episodio
        #num_steps+=1        

        # Parte que termina el episodio si se llega a algun estado terminal
        if done:
            continue_episode = False
            #if state == 'Azul':
                #win_per+=1
                
    # Promedio de pasos por episodio
    prom_step += 1/(i+1) *(num_steps - prom_step)
    #print('-'*20)


In [8]:
Qlearning_policy = {}
for i in es.states:
    #print(i)
    max_val = np.argmax([es.q_values[(i,act)] for act in es.allowed_actions[i]])
    Qlearning_policy[i] = es.allowed_actions[i][max_val]
print(Qlearning_policy)

{1: 'At', 2: 'At', 3: 'At', 4: 'At', 5: 'At', 6: 'At', 7: 'Ad', 8: 'Ad', 9: 'Ad', 10: 'Ad', 11: 'Ad', 12: 'Ad', 13: 'Ad', 14: 'Ad', 15: 'Ad', 16: 'Ad', 17: 'At', 18: 'At', 19: 'At', 20: 'At', 21: 'Ad', 22: 'At', 23: 'Ad', 24: 'Ad', 25: 'Ad', 26: 'Ad', 27: 'Ad', 28: 'At', 29: 'Ad', 30: 'At', 31: 'At', 32: 'At', 33: 'At', 34: 'At', 35: 'At', 36: 'At', 37: 'Ad', 38: 'Ad', 39: 'At', 40: 'At', 41: 'Ad', 42: 'At', 43: 'Ad', 44: 'Ad', 45: 'Ad', 46: 'Ad', 47: 'At', 48: 'Ad', 49: 'Ad', 50: 'Ad', 51: 'Ad', 52: 'Ad', 53: 'At', 54: 'Ad', 55: 'Ad', 56: 'Ad', 57: 'At', 58: 'Ad', 59: 'Ad', 60: 'At', 61: 'At', 62: 'Ad', 63: 'At', 64: 'Ad', 65: 'At', 66: 'Ad', 67: 'At', 68: 'Ad', 69: 'Ad', 70: 'Ad', 71: 'Ad', 72: 'Ad', 73: 'Ad', 74: 'Ad', 75: 'Ad', 76: 'Ad', 77: 'Ad', 78: 'Ad', 79: 'At', 80: 'Ad', 81: 'At', 82: 'At', 83: 'Ad', 84: 'At', 85: 'At', 86: 'At', 87: 'At', 88: 'At', 89: 'Ad', 90: 'Ad', 91: 'Ad', 92: 'Ad', 93: 'Ad', 94: 'Ad', 95: 'Ad', 96: 'At', 97: 'At', 98: 'Ad', 99: 'At', 100: 'Ad'}


In [9]:
# Prueba de polica Qlearning
num_episodios = 1000
win_per = 0
prom_step = 0

for i in range(num_episodios):
    num_steps = 0
    state = es.states[0]
    states = []
    
    continue_episode = True
    while continue_episode:
        # cambia al estado de cima escalera o cola de serpiente
        if state in es.snakes:
            state = es.state_values[es.snakes[state]]
        elif state in es.stairs:
            state = es.state_values[es.stairs[state]]
            
        # Toma la accion de la politica
        action = Qlearning_policy[state]
        # Da un paso en direccion de la politica
        new_state, reward, _, done = es.step(state, action, random=True)
        
        state = new_state
        states.append(state)
        num_steps+=1
        #print(action, state)
        
        if done or num_steps >500:
            continue_episode = False
            if state == 'Azul':
                win_per+=1
    #print(states)
    prom_step += 1/(i+1) *(num_steps - prom_step)
    
#win_per = win_per*100/num_episodios

In [10]:
print('Promedio pasos por episodio: ',prom_step)
print('Porcentaje victorias:' ,win_per*100/num_episodios)

Promedio pasos por episodio:  83.56000000000017
Porcentaje victorias: 90.5
